# Promises

In [1]:
const setTimer = (duration) => {
    const promise = new Promise((resolve, reject) => {
        setTimeout(() => {
            resolve('Done!');
        }, duration);
    });
    return promise;
};

In [7]:
setTimer(1000).then(data => {
    console.log(data);
});

Promise { <pending> }

Done!


## Chaining multiple promises

Chaining multiple promises means executing a sequence of asynchronous tasks one after another, where each task starts after the previous one finishes.

In [8]:
function fetchUserId() {
  return new Promise((resolve) => {
    setTimeout(() => resolve(1), 1000);
  });
}

function fetchUserDetails(userId) {
  return new Promise((resolve) => {
    setTimeout(() => resolve({ id: userId, name: "Alice" }), 1000);
  });
}

function fetchUserPosts(user) {
  return new Promise((resolve) => {
    setTimeout(() => resolve(["Post1", "Post2"]), 1000);
  });
}

In [9]:
// Chaining
fetchUserId()
  .then((id) => {
    console.log("Got ID:", id);
    return fetchUserDetails(id);
  })
  .then((user) => {
    console.log("User details:", user);
    return fetchUserPosts(user);
  })
  .then((posts) => {
    console.log("User posts:", posts);
  })
  .catch((err) => {
    console.error("Something went wrong:", err);
  });

Promise { <pending> }

Got ID: 1
User details: { id: 1, name: "Alice" }
User posts: [ "Post1", "Post2" ]


In [3]:
const promise2 = new Promise((resolve, reject) => {
    resolve('Second promise done!');
});

In [15]:
promise2
    .then(data => {
        console.log(data);
        return setTimer(1000);
    })
    .then(secondData => {
        console.log(secondData);
    });

Second promise done!


Promise { <pending> }

Done!


⚠️ Error Handling with reject in Promises

In [16]:
function doSomethingRisky() {
  return new Promise((resolve, reject) => {
    const success = false; // Change to true to simulate success

    if (success) {
      resolve("✅ Success! Everything worked.");
    } else {
      reject("❌ Something went wrong!");
    }
  });
}

In [17]:
doSomethingRisky()
  .then((result) => {
    console.log("Result:", result);
  })
  .catch((error) => {
    console.error("Caught an error:", error);
  });

Caught an error: ❌ Something went wrong!


Promise { undefined }

In [18]:
function fetchUserId() {
  return new Promise((resolve, reject) => {
    setTimeout(() => {
      console.log("Fetching user ID...");
      const success = false; // Change to true to avoid rejection
      if (success) {
        resolve(1);
      } else {
        reject("Failed to fetch user ID");
      }
    }, 1000);
  });
}

function fetchUserDetails(userId) {
  return new Promise((resolve, reject) => {
    setTimeout(() => {
      console.log("Fetching user details...");
      resolve({ id: userId, name: "Alice" });
    }, 1000);
  });
}

function fetchUserPosts(user) {
  return new Promise((resolve, reject) => {
    setTimeout(() => {
      console.log("Fetching user posts...");
      resolve(["Post1", "Post2"]);
    }, 1000);
  });
}

In [20]:
// Chaining with error handling
fetchUserId()
  .then((id) => fetchUserDetails(id))
  .then((user) => fetchUserPosts(user))
  .then((posts) => {
    console.log("Posts:", posts);
  })
  .catch((err) => {
    console.error("❌ Error caught:", err);
  });

Promise { <pending> }

Fetching user ID...


❌ Error caught: Failed to fetch user ID


In [29]:
function fetchUserId() {
  return new Promise((resolve, reject) => {
    setTimeout(() => {
      console.log("Fetching user ID...");
      const success = true; // Change to true to avoid rejection
      if (success) {
        resolve(1);
      } else {
        reject("Failed to fetch user ID");
      }
    }, 1000);
  });
}

function fetchUserDetails(userId) {
  return new Promise((resolve, reject) => {
    setTimeout(() => {
      console.log("Fetching user details...");
      const success = false; // Change to true to avoid rejection
      if (success) {
        resolve({ id: userId, name: "Alice" });
      } else {
        reject("Failed to fetch user details");
      }
    }, 1000);
  });
}

function fetchUserPosts(user) {
  return new Promise((resolve, reject) => {
    setTimeout(() => {
      console.log("Fetching user posts...");
      resolve(["Post1", "Post2"]);
    }, 1000);
  });
}

## ✅ Correct Global Error Handling

✅ This catches any error that occurs in the entire chain above it (from fetchUserId all the way to fetchUserPosts).

In [27]:
// Chaining with error handling
fetchUserId()
  .then((id) => fetchUserDetails(id))
  .then((user) => fetchUserPosts(user))
  .then((posts) => {
    console.log("Posts:", posts);
  })
  .catch((err) => {
    console.error("❌ Error caught:", err);
  });

Promise { <pending> }

Fetching user ID...
Fetching user details...


❌ Error caught: Failed to fetch user details


## ⚠️ Incorrect / Partial Error Handling

⚠️ This catches errors only from fetchUserId() or fetchUserDetails(), and then continues to run fetchUserPosts() — even though the user may be undefined.

This version is less safe, because it continues the chain even after an error. It doesn't "stop" or prevent further .then()s from running.

In [31]:
// Chaining with error handling
fetchUserId()
  .then((id) => fetchUserDetails(id))
  .catch((err) => {
    console.error("❌ Error caught:", err);
  })
  .then((user) => fetchUserPosts(user))
  .then((posts) => {
    console.log("Posts:", posts);
  });

Promise { <pending> }

Fetching user ID...
Fetching user details...


❌ Error caught: Failed to fetch user details


Fetching user posts...
Posts: [ "Post1", "Post2" ]


You learned about the different promise states:

PENDING => Promise is doing work, neither then() nor catch() executes at this moment

RESOLVED => Promise is resolved => then() executes

REJECTED  => Promise was rejected => catch() executes

When you have another then() block after a catch() or then() block, the promise re-enters PENDING mode (keep in mind: then() and catch() always return a new promise - either not resolving to anything or resolving to what you return inside of then()). Only if there are no more then() blocks left, it enters a new, final mode: SETTLED.

Once SETTLED, you can use a special block - finally() - to do final cleanup work. finally() is reached no matter if you resolved or rejected before.

In [32]:
fetchUserId()
  .then((id) => fetchUserDetails(id))
  .catch((err) => {
    console.error("❌ Error caught:", err);
  })
  .finally(() => {
    console.log('finally!');
  });

Promise { <pending> }

Fetching user ID...
Fetching user details...


❌ Error caught: Failed to fetch user details


finally!


In [33]:
fetchUserId()
  .then((id) => {})
  .catch((err) => {
    console.error("❌ Error caught:", err);
  })
  .finally(() => {
    console.log('finally!');
  });

Promise { <pending> }

Fetching user ID...
finally!


## 🔁 Callback Hell

Callback Hell (also known as "Pyramid of Doom") happens when you use nested callbacks, making your code deeply indented, hard to read, and harder to maintain.

In [2]:
function getUserId(callback) {
  setTimeout(() => {
    console.log("Fetched user ID");
    callback(1);
  }, 1000);
}

function getUserDetails(userId, callback) {
  setTimeout(() => {
    console.log("Fetched user details for ID:", userId);
    callback({ id: userId, name: "Alice" });
  }, 1000);
}

function getUserPosts(user, callback) {
  setTimeout(() => {
    console.log("Fetched posts for user:", user.name);
    callback(["Post1", "Post2"]);
  }, 1000);
}

In [3]:
// ❌ Callback Hell
getUserId(function (id) {
  getUserDetails(id, function (user) {
    getUserPosts(user, function (posts) {
      console.log("Posts:", posts);
      // Imagine more nested callbacks here...
    });
  });
});

Fetched user ID
Fetched user details for ID: 1
Fetched posts for user: Alice
Posts: [ "Post1", "Post2" ]


## 🏁 Promise.race()

Returns the first promise to settle (resolve or reject).

In [4]:
const fast = new Promise((resolve) => setTimeout(() => resolve("Fast!"), 100));
const slow = new Promise((resolve) => setTimeout(() => resolve("Slow..."), 500));

In [5]:
Promise.race([fast, slow]).then((result) => {
  console.log("🏁 Winner:", result);
});

🏁 Winner: Fast!


Promise { undefined }

## ✅ Promise.all()

Waits for all promises to resolve. If any promise rejects, it fails immediately.

In [6]:
const p1 = Promise.resolve("A");
const p2 = Promise.resolve("B");
const p3 = Promise.resolve("C");

In [7]:
Promise.all([p1, p2, p3])
  .then((results) => {
    console.log("✅ All resolved:", results); // ['A', 'B', 'C']
  })
  .catch((err) => {
    console.error("❌ One failed:", err);
  });

✅ All resolved: [ "A", "B", "C" ]


Promise { undefined }

In [8]:
const p1 = Promise.resolve("A");
const p2 = Promise.reject("B failed");

In [9]:
Promise.all([p1, p2])
  .then(console.log)
  .catch((err) => console.error("❌ Error:", err));

❌ Error: B failed


Promise { undefined }

## 🚦 Promise.allSettled()

Waits for all promises to finish, regardless of resolve or reject. Returns an array of result objects.

In [10]:
const p1 = Promise.resolve("Success!");
const p2 = Promise.reject("Failure!");
const p3 = Promise.resolve("Another success!");

In [11]:
Promise.allSettled([p1, p2, p3]).then((results) => {
  console.log("📋 Results:", results);
});

📋 Results: [
  { status: "fulfilled", value: "Success!" },
  { status: "rejected", reason: "Failure!" },
  { status: "fulfilled", value: "Another success!" }
]


Promise { undefined }

## 🔍 Promise.any()

Waits for any promise to resolve. Ignores rejections unless all fail.

In [12]:
const p1 = Promise.reject("Fail 1");
const p2 = Promise.reject("Fail 2");
const p3 = Promise.resolve("🎉 First success");

In [13]:
Promise.any([p1, p2, p3])
  .then((value) => {
    console.log("✅ First resolved:", value);
  })
  .catch((err) => {
    console.error("❌ All failed:", err);
  });

✅ First resolved: 🎉 First success


Promise { undefined }